In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [2]:
spark = SparkSession.builder.getOrCreate()

2023-02-24 19:59:51,090 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df = spark.read.format("csv").option("header", True).load("file:///home/train/datasets/order_status/")

In [4]:
df.limit(5).toPandas()

,ORDER_ID,SUBSCRIBER_ID,STATUS_DATE,STATUS_TIME,STATUS
0,100159,200427,20230223,83209,ASSIGNED
1,100159,200427,20230223,74232,RETURNED
2,100159,200427,20230222,95056,CREATED
3,100410,200366,20230223,91017,ASSIGNED
4,100410,200366,20230223,30301,RETURNED


In [5]:
df.printSchema()

root
 |-- ORDER_ID: string (nullable = true)
 |-- SUBSCRIBER_ID: string (nullable = true)
 |-- STATUS_DATE: string (nullable = true)
 |-- STATUS_TIME: string (nullable = true)
 |-- STATUS: string (nullable = true)



In [6]:
df.groupBy("STATUS").count().orderBy(F.desc("count")).show()

+---------+-----+
|   STATUS|count|
+---------+-----+
|  CREATED|  586|
| RETURNED|  346|
|COMPLETED|  290|
| ASSIGNED|  165|
|     POOL|  100|
|CANCELLED|   31|
+---------+-----+



# Timestamp format

In [7]:
df2 = df.withColumn("TS2", F.concat(F.col("STATUS_DATE"), F.lit(" "), F.col("STATUS_TIME"))) \
.withColumn("TS", F.to_timestamp(F.col("TS2"), "yyyyMMdd Hmmss"))

df2.limit(5).toPandas()

,ORDER_ID,SUBSCRIBER_ID,STATUS_DATE,STATUS_TIME,STATUS,TS2,TS
0,100159,200427,20230223,83209,ASSIGNED,20230223 83209,2023-02-23 08:32:09
1,100159,200427,20230223,74232,RETURNED,20230223 74232,2023-02-23 07:42:32
2,100159,200427,20230222,95056,CREATED,20230222 95056,2023-02-22 09:50:56
3,100410,200366,20230223,91017,ASSIGNED,20230223 91017,2023-02-23 09:10:17
4,100410,200366,20230223,30301,RETURNED,20230223 30301,2023-02-23 03:03:01


In [8]:
df3 = df2.drop("TS2")

df3.limit(5).toPandas()

,ORDER_ID,SUBSCRIBER_ID,STATUS_DATE,STATUS_TIME,STATUS,TS
0,100159,200427,20230223,83209,ASSIGNED,2023-02-23 08:32:09
1,100159,200427,20230223,74232,RETURNED,2023-02-23 07:42:32
2,100159,200427,20230222,95056,CREATED,2023-02-22 09:50:56
3,100410,200366,20230223,91017,ASSIGNED,2023-02-23 09:10:17
4,100410,200366,20230223,30301,RETURNED,2023-02-23 03:03:01


# CREATE DATE END DATE

# Window

In [14]:
df4 = df3.withColumn("order_min_ts", F.min("TS").over(Window.partitionBy("ORDER_ID")))  \
         .withColumn("order_max_ts", F.max("TS").over(Window.partitionBy("ORDER_ID"))) \
         .withColumn("rn", F.row_number().over(Window.partitionBy("ORDER_ID").orderBy(F.desc("TS")))) \
         .orderBy("ORDER_ID", "rn")

df4.limit(5).toPandas()

,ORDER_ID,SUBSCRIBER_ID,STATUS_DATE,STATUS_TIME,STATUS,TS,order_min_ts,order_max_ts,rn
0,100001,200574,20230223,81438,ASSIGNED,2023-02-23 08:14:38,2023-02-22 09:53:06,2023-02-23 08:14:38,1
1,100001,200574,20230223,74734,RETURNED,2023-02-23 07:47:34,2023-02-22 09:53:06,2023-02-23 08:14:38,2
2,100001,200574,20230222,95306,CREATED,2023-02-22 09:53:06,2023-02-22 09:53:06,2023-02-23 08:14:38,3
3,100002,200121,20230222,93712,POOL,2023-02-22 09:37:12,2023-02-22 09:37:12,2023-02-22 09:37:12,1
4,100002,200121,20230222,93712,CREATED,2023-02-22 09:37:12,2023-02-22 09:37:12,2023-02-22 09:37:12,2


In [15]:
df5 = df4.withColumn("START_DATE", F.col("order_min_ts")) \
         .withColumn("END_DATE", F.when(F.col("STATUS").isin("COMPLETED", "CANCELLED"), 
                               F.col("order_max_ts"))).where("rn = 1")


df5.limit(20).toPandas()

,ORDER_ID,SUBSCRIBER_ID,STATUS_DATE,STATUS_TIME,STATUS,TS,order_min_ts,order_max_ts,rn,START_DATE,END_DATE
0,100001,200574,20230223,81438,ASSIGNED,2023-02-23 08:14:38,2023-02-22 09:53:06,2023-02-23 08:14:38,1,2023-02-22 09:53:06,NaT
1,100002,200121,20230222,93712,POOL,2023-02-22 09:37:12,2023-02-22 09:37:12,2023-02-22 09:37:12,1,2023-02-22 09:37:12,NaT
2,100003,200432,20230223,82249,ASSIGNED,2023-02-23 08:22:49,2023-02-22 09:40:01,2023-02-23 08:22:49,1,2023-02-22 09:40:01,NaT
3,100004,200234,20230222,104426,COMPLETED,2023-02-22 10:44:26,2023-02-22 09:37:58,2023-02-22 10:44:26,1,2023-02-22 09:37:58,2023-02-22 10:44:26
4,100005,200546,20230222,94727,POOL,2023-02-22 09:47:27,2023-02-22 09:47:27,2023-02-22 09:47:27,1,2023-02-22 09:47:27,NaT
5,100006,200369,20230223,91333,ASSIGNED,2023-02-23 09:13:33,2023-02-22 10:54:13,2023-02-23 09:13:33,1,2023-02-22 10:54:13,NaT
6,100007,200486,20230223,91359,ASSIGNED,2023-02-23 09:13:59,2023-02-22 10:54:48,2023-02-23 09:13:59,1,2023-02-22 10:54:48,NaT
7,100008,200190,20230223,83004,ASSIGNED,2023-02-23 08:30:04,2023-02-22 09:43:31,2023-02-23 08:30:04,1,2023-02-22 09:43:31,NaT
8,100009,200058,20230222,195548,COMPLETED,2023-02-22 19:55:48,2023-02-22 09:49:51,2023-02-22 19:55:48,1,2023-02-22 09:49:51,2023-02-22 19:55:48
9,100010,200253,20230222,120725,COMPLETED,2023-02-22 12:07:25,2023-02-22 09:43:01,2023-02-22 12:07:25,1,2023-02-22 09:43:01,2023-02-22 12:07:25


In [16]:
df5.printSchema()

root
 |-- ORDER_ID: string (nullable = true)
 |-- SUBSCRIBER_ID: string (nullable = true)
 |-- STATUS_DATE: string (nullable = true)
 |-- STATUS_TIME: string (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- TS: timestamp (nullable = true)
 |-- order_min_ts: timestamp (nullable = true)
 |-- order_max_ts: timestamp (nullable = true)
 |-- rn: integer (nullable = true)
 |-- START_DATE: timestamp (nullable = true)
 |-- END_DATE: timestamp (nullable = true)



In [17]:
df6 = df5.withColumn("DURATION", F.when(F.col("END_DATE").isNull(), 0) \
                        .otherwise(  (F.col("END_DATE").cast(LongType()) - F.col("START_DATE").cast(LongType())) / 3600  )) \
.drop("STATUS_DATE", "STATUS_TIME", "TS", "order_min_ts", "order_max_ts", "rn")

#  - F.col("START_DATE")

In [18]:
df6.limit(50).toPandas()

,ORDER_ID,SUBSCRIBER_ID,STATUS,START_DATE,END_DATE,DURATION
0,100001,200574,ASSIGNED,2023-02-22 09:53:06,NaT,0.000000
1,100002,200121,POOL,2023-02-22 09:37:12,NaT,0.000000
2,100003,200432,ASSIGNED,2023-02-22 09:40:01,NaT,0.000000
3,100004,200234,COMPLETED,2023-02-22 09:37:58,2023-02-22 10:44:26,1.107778
4,100005,200546,POOL,2023-02-22 09:47:27,NaT,0.000000
5,100006,200369,ASSIGNED,2023-02-22 10:54:13,NaT,0.000000
6,100007,200486,ASSIGNED,2023-02-22 10:54:48,NaT,0.000000
7,100008,200190,ASSIGNED,2023-02-22 09:43:31,NaT,0.000000
8,100009,200058,COMPLETED,2023-02-22 09:49:51,2023-02-22 19:55:48,10.099167
9,100010,200253,COMPLETED,2023-02-22 09:43:01,2023-02-22 12:07:25,2.406667


In [19]:
df6.printSchema()

root
 |-- ORDER_ID: string (nullable = true)
 |-- SUBSCRIBER_ID: string (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- START_DATE: timestamp (nullable = true)
 |-- END_DATE: timestamp (nullable = true)
 |-- DURATION: double (nullable = true)



In [ ]:
spark.stop()